In [29]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob
  
class TwitterClient(object): 
    ''' 
	Generic Twitter Class for sentiment analysis. 
	'''
    def __init__(self): 
        ''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console
        consumer_key = 'AAtK8anyeQi2lQhV7eJRAON51'
        consumer_secret = 'MhpgXKrmob91fn0YZkVpBfWW5wPMIhAszAtc7YNU5KHXbP7pp1'
        access_token = '2491372235-QfIq9SyAOgA3FqWIqkAtpLnmgdaURyCakcWsfB5'
        access_token_secret = 'CRGtQYQ7MuegCKpatN2JMrac5wEw4eGksyALCRcyIzLgu'
        # attempt authentication
        try:
            # create OAuthHandler object  
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet):
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet):
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets 
  
        except tweepy.TweepError as e:
             # print error (if any) 
            print("Error : " + str(e)) 
  
def main(): 
    # creating object of TwitterClient Class
    api = TwitterClient() 
    # calling function to get tweets
    tweets = api.get_tweets(query = 'Donald Trump', count = 200)
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
   
   
if __name__ == "__main__":
    # calling main function 
    main() 

Positive tweets percentage: 33 %
Negative tweets percentage: 27 %


Positive tweets:
RT @jdmaccoby: [Donald Trump, Titanic captain]

1. There is no iceberg 
2. We won’t hit the iceberg
3. We barely touched the iceberg
4. Nob…
RT @GotCashed: Did you hear that Nancy Pelosi has created a “special” oversight committee, more “special” than the existing oversight commi…
RT @votevets: Real leaders stand up for the lives of the men and women who serve under them. Thank you, Capt. Crozier, for being a true lea…
RT @JoeBiden: This is a headline every American should see: “Trump administration ended pandemic early-warning program to detect coronaviru…
RT @WalshFreedom: I can think of no single event that better sums up the awfulness of the Donald Trump Presidency than a Navy Captain being…
RT @Weinsteinlaw: Has Donald Trump expressed a single word of sympathy for the thousands of American families who’ve lost a loved one to th…
RT @amyklobuchar: So it’s ok for Donald Trump &amp; 1st Lady but he o